# <center>Threading

In [1]:
import time
import shutil
import requests
import threading
import multiprocessing
import concurrent.futures
from pathlib import Path

In [2]:
def sleep_fun():
    print("I'm starting sleep_fun")
    print(threading.current_thread().name)
    time.sleep(2)
    print("I've finished sleep_fun")

sleep_fun()
print("I'm in the main program")

I'm starting sleep_fun
MainThread
I've finished sleep_fun
I'm in the main program


In [3]:
t1 = threading.Thread(target=sleep_fun)
t1.start()
print(threading.current_thread().name)
print("I'm in the main program")

I'm starting sleep_fun
Thread-6
MainThread
I'm in the main program


In [4]:
t1 = threading.Thread(target=sleep_fun)
t1.start()
t1.join()
print(threading.current_thread().name)
print("I'm in the main program")

I'm starting sleep_fun
Thread-7
I've finished sleep_fun
I've finished sleep_fun
MainThread
I'm in the main program


In [5]:
t1 = threading.Thread(target=sleep_fun, daemon=True)
t1.start()
t1.join()
print(threading.current_thread().name)
print("I'm in the main program")

I'm starting sleep_fun
Thread-8
I've finished sleep_fun
MainThread
I'm in the main program


In [54]:
def fact(num):
    start = time.time()
    x = 1 
    for i in range(1, num):
        x *= i
    time.sleep(1)
    stop = time.time() - start
    print(f'time elapsed {stop}')

In [7]:
start = time.time()
for i in range(3):
    fact(50000)
stop = time.time() - start
print(f'Total time elapsed is {stop}')

time elapsed 4.281245708465576
time elapsed 4.249553442001343
time elapsed 4.297409772872925
Total time elapsed is 12.828208923339844


In [8]:
threads = []
num_threads = 3
for i in range(num_threads):
    print(f'registering thread {i}')
    t = threading.Thread(target=fact, args=(50000,))
    threads.append(t)

registering thread 0
registering thread 1
registering thread 2


In [9]:
start = time.time()
for t in threads:
    t.start()
for t in threads:
    t.join()
stop = time.time() - start
print(f'Total time elapsed is {stop}')

time elapsed 11.375181913375854
time elapsed 11.921014308929443
time elapsed 12.531141996383667
Total time elapsed is 12.578321933746338


In [10]:
start = time.time()
with concurrent.futures.ThreadPoolExecutor() as executer:
    results = [executer.submit(fact, 50000) for i in range(3)]
stop = time.time() - start
print(f'Total time elapsed is {stop}')

time elapsed 12.234408378601074
time elapsed 12.234368085861206
time elapsed 12.449917316436768
Total time elapsed is 12.578134775161743


In [11]:
photos = [
    'https://unsplash.com/photos/LxaorEDmI3c/download?force=true',
    'https://unsplash.com/photos/4rDCa5hBlCs/download?force=true',
    'https://unsplash.com/photos/jFCViYFYcus/download?force=true',
    'https://unsplash.com/photos/EwKXn5CapA4/download?force=true',
    'https://unsplash.com/photos/1Z2niiBPg5A/download?force=true',
    'https://unsplash.com/photos/G15G-Any-D0/download?force=true',
    'https://unsplash.com/photos/01_igFr7hd4/download?force=true',
    'https://unsplash.com/photos/78A265wPiO4/download?force=true',
    'https://unsplash.com/photos/tGTVxeOr_Rs/download?force=true',
    'https://unsplash.com/photos/hFzIoD0F_i8/download?force=true'
]

In [12]:
response = requests.get(photos[0])

In [18]:
fname = response.headers['Content-Disposition'].split('filename=')[-1].replace('"', '')

In [19]:
with open(fname, 'wb') as fs:
    fs.write(response.content)

In [23]:
def download_image(image):
    response = requests.get(image)
    fname = response.headers['Content-Disposition'].split('filename=')[-1].replace('"', '')
    Path('./photos').mkdir(exist_ok=True)
    fname = './photos/' + fname
    with open(fname, 'wb') as fs:
        fs.write(response.content)

In [24]:
start = time.time()
r = map(download_image, photos)
for i in r:
    pass
stop = time.time() - start
print(f'Total elapsed time {stop}')

Total elapsed time 146.40641927719116


In [25]:
start = time.time()
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executer:
    executer.map(download_image, photos)
stop = time.time() - start
print(f'Total elapsed time {stop}')

Total elapsed time 86.09387850761414


In [26]:
start = time.time()
with concurrent.futures.ThreadPoolExecutor() as executer:
    executer.map(download_image, photos)
stop = time.time() - start
print(f'Total elapsed time {stop}')

Total elapsed time 39.12469410896301


In [28]:
files = list(Path('./photos').glob('*.*'))

In [29]:
files

[WindowsPath('photos/boxed-water-is-better-G15G-Any-D0-unsplash.jpg'),
 WindowsPath('photos/boxed-water-is-better-LxaorEDmI3c-unsplash.jpg'),
 WindowsPath('photos/casey-horner-4rDCa5hBlCs-unsplash.jpg'),
 WindowsPath('photos/david-marcu-78A265wPiO4-unsplash.jpg'),
 WindowsPath('photos/jeremy-bishop-EwKXn5CapA4-unsplash.jpg'),
 WindowsPath('photos/luca-bravo-hFzIoD0F_i8-unsplash.jpg'),
 WindowsPath('photos/lukasz-szmigiel-jFCViYFYcus-unsplash.jpg'),
 WindowsPath('photos/niko-photos-tGTVxeOr_Rs-unsplash.jpg'),
 WindowsPath('photos/qingbao-meng-01_igFr7hd4-unsplash.jpg'),
 WindowsPath('photos/v2osk-1Z2niiBPg5A-unsplash.jpg')]

In [32]:
def copy_files(file):
    fname = file.name
    shutil.copy(Path('./photos').joinpath(fname), Path('./photos-new').joinpath(fname))

In [33]:
Path('./photos-new').mkdir(exist_ok=True)
start = time.time()
with concurrent.futures.ThreadPoolExecutor() as executer:
    executer.map(copy_files, files)
stop = time.time() - start
print(f'Total elapsed time {stop}')

Total elapsed time 0.07919478416442871


In [34]:
class Course:
    def __init__(self):
        self.num_student = 0
        
    def add_student(self, name):
        print(f'Start adding student --> Thread {name}')
        my_num_student = self.num_student
        my_num_student += 1
        time.sleep(0.1)
        self.num_student = my_num_student
        print(f'Finished adding student --> Thread {name}')

In [36]:
cs1 = Course()
print(f'Testing adding student initial number of students {cs1.num_student}')
with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executer:
    for i in range(2):
        executer.submit(cs1.add_student, i)
print(f'Testing adding student initial number of students {cs1.num_student}')

Testing adding student initial number of students 0
Start adding student --> Thread 0
Start adding student --> Thread 1
Finished adding student --> Thread 1Finished adding student --> Thread 0

Testing adding student initial number of students 1


In [ ]:
class Course:
    def __init__(self):
        self.num_student = 0
        
    def add_student(self, name, lock):
        print(f'Start adding student --> Thread {name}')
        with lock:
            my_num_student = self.num_student
            my_num_student += 1
            time.sleep(0.1)
            self.num_student = my_num_student
        print(f'Finished adding student --> Thread {name}')

In [38]:
cs1 = Course()
lock = threading.Lock()
print(f'Testing adding student initial number of students {cs1.num_student}')
with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executer:
    for i in range(2):
        executer.submit(cs1.add_student, i, lock)
print(f'Testing adding student initial number of students {cs1.num_student}')

Testing adding student initial number of students 0
Start adding student --> Thread 0
Start adding student --> Thread 1
Finished adding student --> Thread 0
Finished adding student --> Thread 1
Testing adding student initial number of students 2


# <center>Processes

In [68]:
processes = []
num_processes = 3
for i in range(num_processes):
    print(f'registering process {i}')
    p = multiprocessing.Process(target=fact, args=(50000,))
    processes.append(p)

registering process 0
registering process 1
registering process 2


In [69]:
start = time.time()
for p in processes:
    p.start()
for p in processes:
    p.join()
stop = time.time() - start
print(f'Total time elapsed is {stop}')

Total time elapsed is 0.2504284381866455


## [Array data types](https://docs.python.org/3/library/array.html)